<a href="https://colab.research.google.com/github/andrew-veriga/dlaicourse/blob/master/TensorFlow%20Deployment/Course%201%20-%20TensorFlow-JS/Week%203/Exercise/TFJS_Week3_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Преобразование модели Keras в формат JSON

На предыдущем уроке вы узнали, как использовать CNN, чтобы сделать распознавание рукописных цифр более эффективным. В этом уроке вы перейдете на следующий уровень, распознавая реальные изображения кошек и собак, чтобы классифицировать входящее изображение как одно или другое. В частности, распознавание рукописного ввода сделало вашу жизнь немного проще, поскольку все изображения были одного размера и формы, и все они были монохромными. Реальные изображения не такие - они разных форм, соотношений сторон и т.д., И, кроме того, обычно они цветные.

Как часть задачи вам необходимо обработать данные - изменить их размер, чтобы они были одинаковыми по форме.

Вы выполните следующие действия:

1. Изучите пример данных о кошках и собаках.
2. Построете и обучите нейронную сеть, их различающую.
3. Оцените точность обучения и проверки.
4. Сохраните обученную модель как файл Keras HDF5.
5. При помощи конвертера tensorflow.js, преобразуете сохраненную модель Keras в формат JSON.


# Импорт ресурсов

Чтобы использовать преобразователь tensorflow.js, нам необходимо установить `tensorflowjs`.

In [ ]:
!pip install tensorflowjs

In [ ]:
import tensorflow as tf

print('\u2022 Using TensorFlow Version:', tf.__version__)

## Изучите примеры данных

Начнем с загрузки данных из нашего примера, архива .zip с изображениями кошек и собак в формате JPG и его распаковки в локальный каталог `/ tmp`.

**Примечание:**  2000 изображений, используемых в этом упражнении, взяты из [набора данных «Собаки против кошек»] (https://www.kaggle.com/c/dogs-vs-cats/data), доступного на Kaggle, который содержит 25 000 изображений. Здесь мы используем подмножество полного набора данных, чтобы сократить время обучения в образовательных целях.

In [ ]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O /tmp/cats_and_dogs_filtered.zip

Следующий код Python будет использовать библиотеку OS, предоставляющую  доступ к файловой системе, и библиотеку zipfile, позволяющую распаковать данные.

In [ ]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

Содержимое .zip извлекается в базовый каталог `/tmp/cats_and_dogs_filtered`, который содержит подкаталоги` train` и `validation` для наборов данных обучения и проверки, каждый из которых, в свою очередь, содержит подкаталоги "cats" и "dogs".

В этом примере следует вспомнить: мы не маркируем изображения явно как кошек или собак. В примере с рукописным вводом ранее мы пометили «это 1», «это 7» и т.д. Здесь они автоматически размечиваются по имени своих подкаталогов. ImageGenerator размечает изображения соответствующим образом, сокращая этап кодирования.

Определим эти каталоги:

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat/dog pictures
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat/dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')



Теперь давайте посмотрим, как выглядят имена файлов в каталогах `cats` и` dogs`, `train` (соглашения об именах файлов в каталоге` validation` одинаковы):

In [ ]:
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

Узнаем общее количество изображений кошек и собак в каталогах `train` и `validation`:

In [ ]:
print('total training cat images :', len(os.listdir(      train_cats_dir ) ))
print('total training dog images :', len(os.listdir(      train_dogs_dir ) ))

print('total validation cat images :', len(os.listdir( validation_cats_dir ) ))
print('total validation dog images :', len(os.listdir( validation_dogs_dir ) ))

И для кошек, и для собак у нас есть 1000 обучающих изображений и 500 проверочных изображений.

Теперь выведем несколько изображений, чтобы лучше понять, как выглядят наборы данных о кошках и собаках. Сначала мы настраиваем параметры `matplotlib`:

In [ ]:
%matplotlib inline

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 4
ncols = 4

pic_index = 0 # Index for iterating over images

Отобразим серию из 8 изображений кошек и 8 собак. Вы можете повторно запускать ячейку, чтобы каждый раз видеть новую партию:

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

pic_index+=8

next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
    # Set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1)
    sp.axis('Off') # Don't show axes (or gridlines)
    
    img = mpimg.imread(img_path)
    plt.imshow(img)

plt.show()


Существенное отличие от предыдущего урока в том, что эти изображения бывают разных форм и размеров. Когда вы делали пример распознавания рукописного ввода, вам приходилось работать с изображениями в оттенках серого 28x28. Здесь они бывают цветными и разные по форме и размерам. Перед обучением нейронной сети с ними вам необходимо настроить изображения. Вы увидите это в следующем разделе.

Теперь, когда у вас есть представление о том, как выглядят ваши данные, следующим шагом будет создание модели, которая будет обучена распознавать кошек или собак по этим изображениям.

## Создание небольшой модели с нуля для достижения точности ~ 72%

В предыдущем разделе вы видели, что изображения разных форм и размеров. Чтобы обучить нейронную сеть обрабатывать их, нужно сделать их одного размера. Здесь мы выбрали размер 150x150, и вскоре вы увидите код, который предварительно обрабатывает изображения до этой формы.

Но прежде чем мы продолжим, давайте приступим к созданию модели. Мы определим Sequential, как и раньше, добавив в его начале несколько сверточных слоев. 
Обратите внимание: поскольку здесь у нас задача классификации на два класса, то есть задача *двоичной классификации*, в последнем слое сети мы поставим функцию активации [*sigmoid* активации](https://wikipedia.org/wiki/Sigmoid_function), чтобы на выходе нашей сети был один скаляр от 0 до 1, кодирующим вероятность того, что текущее изображение относится к классу 1 (а не классу 0).

In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'), 
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [ ]:
model.summary()


Столбец «output shape» показывает, как размер вашего признак-вектора изменяется на каждом последующем слое. Слои свертки немного уменьшают размер признак-вектора из-за паддинга, а каждый слой пулинга уменьшает размеры вдвое.

Далее мы настроим спецификации для обучения модели. Мы будем обучать нашу модель с функцией потерь `binary_crossentropy`, потому что это задача двоичной классификации, и наша последняя активация - `sigmoid`. (Чтобы узнать больше о функциях потерь, см. [Ускоренный курс машинного обучения](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) Мы будем использовать оптимизатор `rmsprop` со скоростью обучения 0,001. Во время обучения нам нужно будет следить за точностью классификации.

**Примечание**: В этом случае использование [RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) прдпочтительнее, чем [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), потому что RMSprop автоматизирует настройку скорости обучения. (Дрпугие оптимизаторы, такие как [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) и [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), также автоматически адаптируют скорость обучения во время обучения и будут здесь хорошо работать)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

### Препроцессинг данных

Установим генераторы данных, которые будут считывать изображения из наших исходных папок, преобразовывать их в тензоры `float32` и передавать их (вместе с их метками) в нашу сеть. У нас будет один генератор для обучающих изображений и один для тестовых. Наши генераторы будут формировать пакеты из 20 изображений размером 150x150 и их меток (двоичных).

Как вы, возможно, уже знаете, данные, которые поступают в нейронные сети, обычно должны быть нормализованы каким-то образом, чтобы сделать их более доступными для обработки в сети. (Необработанные пиксели редко загружаются в сверточные сети.) В нашем случае мы будем предварительно обрабатывать наши изображения, нормализуя значения пикселей, чтобы они находились в диапазоне [0, 1] (изначально все значения находятся в диапазоне [0, 255] ).

В Keras это можно сделать с помощью класса `keras.preprocessing.image.ImageDataGenerator` с помощью параметра `rescale`. Этот класс `ImageDataGenerator` позволяет вам создавать экземпляры генераторов аугментированных пакетов изображений (и их меток) через .flow (data, labels) или .flow_from_directory (directory). Эти генераторы затем можно использовать с методами модели Keras, которые принимают генераторы данных в качестве входных данных: `fit`, ʻevaluate_generator` и` pred_generator`.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))


### Тренировка
Let's train on all 2,000 images available, for 15 epochs, and validate on all 1,000 test images. (This may take a few minutes to run.)

Do note the values per epoch.

You'll see 4 values per epoch -- Loss, Accuracy, Validation Loss and Validation Accuracy. 

Давайте потренируемся на всех 2000 доступных изображениях в течение 15 эпох и проверим на всех 1000 тестовых изображениях. (Это может занять несколько минут.)

Обратите внимание на значения в эпохах.

Вы увидите 4 значения для каждой эпохи - потеря, точность, потеря тестирования и точность тестирования.

Loss и Accuracy - отличные показатели прогресса обучения. Делается предположение относительно классификации обучающих данных, которое затем сравнивается с известным значением метки и вычисляет результат. Точность - это часть правильных догадок. Точность тестирования - это измерение с данными, которые не использовались при обучении. Как и ожидалось, она будет немного ниже.

In [ ]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=15,
                              validation_steps=50,
                              verbose=2)

### Оценка точности и потерь для модели

Построим графики точности и потерь обучения/тестирования, собранных во время обучения:

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc, label='Training')
plt.plot  ( epochs, val_acc, label='Validation')
plt.title ('Training and validation accuracy')
plt.legend()
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss, label='Training')
plt.plot  ( epochs, val_loss, label='Validation')
plt.legend()
plt.title ('Training and validation loss')

Как видите, здесь **переобучение**: как будто график вышел из формы. Точность обучения (синим цветом) приближается к 100% (!), А точность проверки (оранжевым цветом) остается на уровне 70%. Наши потери при тестировании достигают минимума всего через пять эпох.

Поскольку у нас относительно небольшое количество обучающих примеров (2000), переобучение должно быть нашей проблемой номер один. Переобучение происходит, когда модель, видевшая слишком мало примеров, обучается шаблонам, которые не обобщаются на новые данные, то есть когда модель начинает использовать нерелевантные признаки для прогнозирования. Например, если вы, как человек, видите только три изображения людей-лесорубов и три изображения людей-моряков, и среди них единственный человек в кепке - лесоруб, вы можете подумать, что в кепке - признак того, что он лесоруб, а не моряк. Тогда из вас получится довольно паршивый классификатор лесорубов / матросов.


Переобучение - центральная проблема в машинном обучении: учитывая, что мы подгоняем параметры нашей модели к заданному набору данных, как мы можем быть уверены, что обобщения, выученные моделью, будут применимы к данным, которых никогда раньше не было? Как нам избежать запоминания вещей, специфичных для обучающих данных?

В следующем упражнении мы рассмотрим способы предотвращения переобучения в модели классификации кошек и собак.

## Сохранение модели

В ячейке ниже сохраните обученную модель как модель Keras.(файл `.h5`).

**Подсказка**: Используйте `model.save()`.
Если не помните, как - загляните в пример `Linear-to-JavaScript.ipynb`.

In [ ]:
# EXERCISE: Save the trained model as a Keras HDF5 file. 

saved_model_path = "./my_model.h5"

# YOUR CODE HERE


## Запустите конвертер TensorFlow.js в сохраненной модели Keras

В ячейке ниже используйте `tensorflowjs_converter`, чтобы преобразовать сохраненную модель Keras в формат JSON.
**ПОДСКАЗКА**: Убедитесь, что вы указали формат входной модели как Keras, используя параметр `--input_format`. Не стесняйтесь взглянуть на пример `Linear-to-JavaScript.ipynb` и в документацию [TensorFlow.js converter documentation](https://github.com/tensorflow/tfjs/tree/master/tfjs-converter#step-1-converting-a-tensorflow-savedmodel-tensorflow-hub-module-keras-hdf5-or-tfkeras-savedmodel-to-a-web-friendly-format).

In [ ]:
# EXERCISE: Use the tensorflow.js converter to convert the saved Keras model into JSON format.

# YOUR CODE HERE


Если вы все сделали правильно, теперь у вас должен быть файл **JSON** с именем `model.json` и несколько файлов `.bin`, таких как `group1-shard1of10.bin`. Количество файлов .bin будет зависеть от размера вашей модели: чем больше ваша модель, тем больше будет файлов .bin. Файл `model.json` содержит архитектуру вашей модели, а файлы `.bin` будут содержать веса вашей модели.